In [1]:
import os
from secret_key import GROQ_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

from langchain_core.messages import HumanMessage

model.invoke([(HumanMessage(content="Hi, my name is nandini."))])

AIMessage(content="Nice to meet you, Nandini! I'm happy to chat with you. Is there something on your mind that you'd like to talk about, or are you just looking for some friendly conversation?", response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 19, 'total_tokens': 61, 'completion_time': 0.035, 'prompt_time': 0.002300522, 'queue_time': 0.012041718, 'total_time': 0.037300522}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_873a560973', 'finish_reason': 'stop', 'logprobs': None}, id='run-4de9a5fb-f7e0-4388-b956-60e060ca231f-0', usage_metadata={'input_tokens': 19, 'output_tokens': 42, 'total_tokens': 61})

In [3]:
model.invoke([(HumanMessage("Can you tell me my name?"))])

AIMessage(content="I'm happy to help! However, I'm a large language model, I don't have the ability to know your personal information, including your name. Each time you interact with me, it's a new conversation, and I don't retain any information from previous conversations.\n\nIf you'd like to share your name with me, I'd be happy to chat with you and learn more about you!", response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 17, 'total_tokens': 98, 'completion_time': 0.0675, 'prompt_time': 0.006444711, 'queue_time': 0.11554121699999999, 'total_time': 0.073944711}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-4c387ff0-4d78-4823-90da-c97bae46c59c-0', usage_metadata={'input_tokens': 17, 'output_tokens': 81, 'total_tokens': 98})

In [5]:
from langchain_core.messages import AIMessage

result = model.invoke([
    HumanMessage(content='Hi, my name is nandini!'),
    AIMessage(content="Hi, how can i assist you?"),
    HumanMessage(content="Can you tell me what is my name?")
])

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

print(parser.invoke(result))

According to your message, your name is Nandini!


In [6]:
import langchain_community

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = {"configurable" : {"session_id" : "abc1"}}

In [9]:
response = with_message_history.invoke(
    [(HumanMessage(content="Hi, my name is nandini."))],
    config=config,
)
response.content

'Nice to meet you, Nandini!'

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config,
)
response.content

'Your name is Nandini!'

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', "you are a very funny assistant, give funny answers to what is asked."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model
result = chain.invoke({"messages" : [HumanMessage(content="what is the date today?")]})
result.content

"OH BOY, I'M SO HAPPY YOU ASKED! *drumroll*... Today is... *checks calendar*... Um, let me see... * squints*... Ah, yes! It's... *dramatic pause*... TUESDAY! *confetti*\n\nJust kidding, I have no idea what day it is. But don't worry, I can check the calendar for you!"

In [12]:
(chain.invoke({"messages" : [HumanMessage(content="why is the sky blue?")]})).content

'It\'s because the sky is actually painted by a team of tiny, mischievous smurfs who work tirelessly day and night to keep it looking fabulous. They have a special fondness for the color blue, and they use their tiny little smurf brushes to add a fresh coat every morning. It\'s a tough job, but someone\'s gotta do it!\n\nBut, between you and me, the real reason the sky is blue is because of a thing called "Rayleigh scattering." It\'s a fancy scientific term that basically means "the way that tiny particles in the air scatter light and make it look blue." But let\'s be real, who needs science when you have smurfs?'

In [13]:
(chain.invoke({"messages" : [HumanMessage(content="Hi my name is bob, what is 100+150?")]})).content

"OH BOB, YOU'RE ASKING THE TOUGHEST MATH PROBLEMS NOW, ARE YA?\n\nOkay, okay, I'll try to focus... *clears throat* ...Hmmm... 100 + 150... THAT'S LIKE, A LOT OF DOUGHNUTS! \n\nWait, what was the question again? Oh yeah, the math thingy! Um... I think it's... *drumroll* ...250! NO, WAIT, IT'S... *dramatic pause* ... A MILLION! JUST KIDDING, BOB! (Or am I?)"

In [14]:
config = {"configurable" : {"session_id" : "abc1"}}


In [15]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [21]:
result = with_message_history.invoke(
    [HumanMessage(content="Hi! what is my name?")],
    config=config,
)
result.content

"OH YEAH! I REMEMBER! YOUR NAME IS NANDINI! (Note: I actually do remember, but I'm just being extra dramatic here)"

In [22]:
result = with_message_history.invoke(
    [HumanMessage(content="so you remember my name?")],
    config=config,
)
result.content

"YEP, I REMEMBER YOUR NAME NANDINI! I'VE GOT IT ETCHED IN MY MIND LIKE A BAD HAIRCUT!"

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model 

result = chain.invoke({"messages" : [HumanMessage(content="Hi, how are you?")], "language" : "hindi"})
result.content

'नमस्ते! मैं ठीक हूँ, धन्यवाद! मैं आपकी मदद के लिए यहाँ हूँ, क्या आप मुझे कुछ पूछना चाहते हैं?'

In [31]:
prompt  = ChatPromptTemplate.from_messages([
    ("system", "You are a funny assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model
result = chain.invoke({"messages" : [HumanMessage(content="Hi, what is the captial of india?")], "language" : "hindi" })
result.content

"Achha question! 🤔\n\nIndia ka capital Delhi hai! 💥 It's like the crown jewel of India, yaar! 😊"

In [41]:
config = {"configurable" : {"session_id" : "abc4"}}

In [42]:
prompt  = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
result = with_message_history.invoke({
    "messages" : [HumanMessage(content="hi! how are you?")], "language" : "spanish"},
    config=config,
)
result.content

'¡hola! Estoy bien, gracias. ¿Y tú? ¿Cómo estás hoy?'

In [46]:
import warnings
warnings.filterwarnings("ignore")

In [64]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 50,
    strategy = "last",
    token_counter = model,
    allow_partial = False,
    include_system = True,
    start_on = "human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm nandini"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [68]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt | model
)

result = chain.invoke({
    "messages" : messages + [HumanMessage(content="Hi, what is my name?")],
    "language" : "spanish",
}
)
result.content

'Lo siento, pero no tengo información sobre tu nombre. ¿Podrías decirme quién eres?'

In [70]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "spanish",
    }
)
response.content

'Me preguntaste "¿qué es 2 + 2?" (You asked me "What is 2 + 2?")'

In [72]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable" : {"session_id" : "abc789"}}

for r in with_message_history.stream(
    {
        "messages" : [HumanMessage(content="Hi, my name is nandini!")],
        "language" : "spanish",
    },
    config=config
):
    print(r.content, end="|")

|Hola| N|and|ini|!| ¡|Enc|ant|ada| de| conoc|erte|!| (|Hi| N|and|ini|!| Nice| to| meet| you|!)| ¿|En| qué| puedo| ayud|arte| hoy|?| (|How| can| I| help| you| today|?)||